In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
devic = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [21]:

train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [22]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [23]:
model = model.to(devic)

In [24]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [25]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(devic)
        kps = kps.to(devic)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14268.3642578125
Epoch 0, iter 10, loss: 15056.220703125
Epoch 0, iter 20, loss: 14621.21875
Epoch 0, iter 30, loss: 13712.619140625
Epoch 0, iter 40, loss: 13821.2587890625
Epoch 0, iter 50, loss: 13295.150390625
Epoch 0, iter 60, loss: 12173.5712890625
Epoch 0, iter 70, loss: 12531.0107421875
Epoch 0, iter 80, loss: 12474.3935546875
Epoch 0, iter 90, loss: 11300.587890625
Epoch 0, iter 100, loss: 11432.765625
Epoch 0, iter 110, loss: 10487.1455078125
Epoch 0, iter 120, loss: 10557.056640625
Epoch 0, iter 130, loss: 9963.7158203125
Epoch 0, iter 140, loss: 9374.435546875
Epoch 0, iter 150, loss: 9660.1806640625
Epoch 0, iter 160, loss: 8624.7294921875
Epoch 0, iter 170, loss: 8336.552734375
Epoch 0, iter 180, loss: 8694.6181640625
Epoch 0, iter 190, loss: 8257.955078125
Epoch 0, iter 200, loss: 7858.76171875
Epoch 0, iter 210, loss: 7930.4765625
Epoch 0, iter 220, loss: 7286.59814453125
Epoch 0, iter 230, loss: 6610.60595703125
Epoch 0, iter 240, loss: 6710.2221

In [27]:
torch.save(model.state_dict(), "keypoints_model.pth")